In [1]:
import numpy as np
import cv2 as cv
from sklearn.metrics import pairwise                                                                                                                                                                                                                                    # Yatharth Jain

In [2]:
background = None
accumulated_weight = 0.5

roi_top = 20
roi_bottom = 300
roi_left = 600
roi_right = 300

In [3]:
def calc_accum_avg(frame, accumulated_weight):
    global background
    if background is None:
        background = frame.copy().astype("float")
        return None
    cv.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
def segment(frame, threshold=25):
    diff = cv.absdiff(background.astype("uint8"), frame)
    ret , thresholded = cv.threshold(diff, threshold, 255, cv.THRESH_BINARY)
    contours, hierarchy = cv.findContours(thresholded.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    else:
        hand_segment = max(contours, key=cv.contourArea)
        return (thresholded, hand_segment)

In [5]:
def count_fingers(threshold, hand_segment):
    conv_hull = cv.convexHull(hand_segment)
    
    top = tuple(conv_hull[conv_hull[:, :, 1].argmin()][0])
    bottom = tuple(conv_hull[conv_hull[:, :, 1].argmax()][0])
    left = tuple(conv_hull[conv_hull[:, :, 0].argmin()][0])
    right = tuple(conv_hull[conv_hull[:, :, 0].argmax()][0])
    cX = (left[0] + right[0]) // 2
    cY = (top[1] + bottom[1]) // 2
    
    distance = pairwise.euclidean_distances([(cX, cY)], Y=[left, right, top, bottom])[0]
    max_distance = distance.max()
    radius = int(0.8 * max_distance)
    circumference = (2 * np.pi * radius)
    
    circular_roi = np.zeros(threshold.shape[:2], dtype="uint8")
    cv.circle(circular_roi, (cX, cY), radius, 255, 10)
    circular_roi = cv.bitwise_and(threshold, threshold, mask=circular_roi)
    countours, hierarchy = cv.findContours(circular_roi.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    
    count = 0
    for ctr in countours:
        (x, y, w, h) = cv.boundingRect(ctr)
        out_of_wrist = (cY + (cY / 3)) > (y + h)
        limit_points = ((circumference / 3) > ctr.shape[0])
        if out_of_wrist and limit_points:
            count += 1
    return count

In [9]:
cam = cv.VideoCapture(0)
num_frame = 0
while True:
    ret,frame = cam.read()
    frame_copy = frame.copy()
    roi=frame[roi_top:roi_bottom, roi_right:roi_left]
    gray = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (7, 7), 0)
    if num_frame < 60:
        calc_accum_avg(gray, accumulated_weight)
        if num_frame <= 59:
            cv.putText(frame_copy, "WAIT!!!", (200, 300), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv.imshow("Finger Count", frame_copy)
    else:
        hand = segment(gray)
        if hand:
            threshold, hand_segment = hand
            cv.drawContours(frame_copy, [hand_segment + (roi_right, roi_top)], -1, (255, 0, 0), 5)
            fingers = count_fingers(threshold, hand_segment)
            cv.putText(frame_copy, str(fingers), (70, 45), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            cv.imshow("Thresholded", threshold)
    cv.rectangle(frame_copy, (roi_left, roi_top), (roi_right, roi_bottom), (0, 0, 255), 5)
    num_frame+=1
    cv.imshow("Finger Count", frame_copy)
    if cv.waitKey(1) & 0xFF == 27:
        break
cam.release()
cv.destroyAllWindows()